In [305]:
import os
import re

import geopandas as gpd
import numpy as np
import pandas as pd
import shapely

from pathlib import Path

In [306]:
data_path = Path(os.environ["DATA_PATH"])
framework_path = Path(os.environ["FRAMEWORK_PATH"])
population_grids_path = Path(os.environ["GRID_PATH"])

In [307]:
df_agebs = gpd.read_file(population_grids_path / "final" / "zone_agebs" / "shaped" / "2020" / "02.2.03.gpkg").to_crs("EPSG:6372")
wanted_cvegeos = set(df_agebs["CVEGEO"].to_numpy().tolist())

In [308]:
lims = gpd.read_file(data_path / "lim_aumxl24/lim_aumxl24.shp").to_crs("EPSG:6372")["geometry"].item()

# Stats

In [323]:
def fillna(col: pd.Series) -> pd.Series:
    return col.replace(["*", "N/D"], np.nan).astype(float)

In [324]:
wanted_cols = ["TVIVPAR", "TVIVPARHAB", "VPH_NDACMM", "VPH_PISOTI", "VPH_REFRI", "VPH_LAVAD", "VPH_HMICRO", "VPH_PC", "VPH_TV", "VPH_INTER", "POBTOT", "POB0_14", "VPH_BICI"]

df_census = (
    pd.read_csv(
        population_grids_path / "initial" / "census" / "INEGI" / "2020" / "conjunto_de_datos_ageb_urbana_02_cpv2020.csv",
        usecols=["ENTIDAD", "MUN", "LOC", "AGEB", "MZA"] + wanted_cols,
    )
)

for col in wanted_cols:
    df_census = df_census.assign(**{col: lambda df: fillna(df[col])})

df_census = (
    df_census
    .query("MZA != 0")
    .assign(
        CVEGEO=lambda df: (
            df["ENTIDAD"].astype(str).str.zfill(2)
            + df["MUN"].astype(str).str.zfill(3)
            + df["LOC"].astype(str).str.zfill(4)
            + df["AGEB"].astype(str).str.zfill(4)
            + df["MZA"].astype(str).str.zfill(3)
        )
    )
    .set_index("CVEGEO")
    .drop(columns=["ENTIDAD", "MUN", "LOC", "AGEB", "MZA"])
)

df_framework = (
    gpd.read_file(framework_path / "2020" / "02_bajacalifornia" / "02m.shp")
    .filter(["geometry", "CVEGEO"])
    .set_index("CVEGEO")
)

joined = (
    df_framework
    .join(df_census, how="inner")
    .assign(CVEGEO_AGEB=lambda df: df.index.str.slice(0, 13))
    .query("CVEGEO_AGEB in @wanted_cvegeos")
    .drop(columns=["CVEGEO_AGEB"])
)

joined = joined[joined.intersects(lims)].to_crs("EPSG:6372")

In [325]:
rows = {
    "total_viviendas_particulares": joined["TVIVPAR"].sum(),
    "frac_viviendas_habitadas": joined["TVIVPARHAB"].sum() / joined["TVIVPAR"].sum(),
    "frac_viviendas_con_vehiculo_privado": 1 - joined["VPH_NDACMM"].sum() / joined["TVIVPARHAB"].sum(),
    "frac_viviendas_con_piso_tierra": joined["VPH_PISOTI"].sum() / joined["TVIVPARHAB"].sum(),
    "frac_viviendas_con_refrigerador": joined["VPH_REFRI"].sum() / joined["TVIVPARHAB"].sum(),
    "frac_viviendas_con_lavadora": joined["VPH_LAVAD"].sum() / joined["TVIVPARHAB"].sum(),
    "frac_viviendas_con_microondas": joined["VPH_HMICRO"].sum() / joined["TVIVPARHAB"].sum(),
    "frac_viviendas_con_computadora": joined["VPH_PC"].sum() / joined["TVIVPARHAB"].sum(),
    "frac_viviendas_con_television": joined["VPH_TV"].sum() / joined["TVIVPARHAB"].sum(),
    "frac_viviendas_con_internet": joined["VPH_INTER"].sum() / joined["TVIVPARHAB"].sum(),
    "frac_viviendas_con_bicicleta": joined["VPH_BICI"].sum() / joined["TVIVPARHAB"].sum(),
}

stats = pd.DataFrame.from_dict(rows, orient="index", columns=["value"]).reset_index(names="stat")
stats

,stat,value
0,total_viviendas_particulares,290570.000000
1,frac_viviendas_habitadas,0.930960
2,frac_viviendas_con_vehiculo_privado,0.809064
3,frac_viviendas_con_piso_tierra,0.004299
4,frac_viviendas_con_refrigerador,0.973265
5,frac_viviendas_con_lavadora,0.883368
6,frac_viviendas_con_microondas,0.707074
7,frac_viviendas_con_computadora,0.542322
8,frac_viviendas_con_television,0.955525
9,frac_viviendas_con_internet,0.745125


# Más lejos de alguna feature

In [326]:
def get_pop_not_intersects(df: gpd.GeoDataFrame, features: gpd.GeoDataFrame, *, pop_col: str) -> float:
    idx_within = df.sjoin(features).index
    blocks_without = joined.loc[~joined.index.isin(idx_within)]
    return blocks_without[pop_col].sum() / df[pop_col].sum()

## Rutas camión

In [327]:
df_routes = (
    gpd.read_file(data_path / "RUTA_TRANSP_URB")
    .filter(["geometry"])
    .to_crs("EPSG:6372")
)
routes_buffered = df_routes.assign(geometry=lambda df: df["geometry"].buffer(500, resolution=64))

rows["frac_personas_mas_lejos_500m_autobus"] = get_pop_not_intersects(joined, routes_buffered, pop_col="POBTOT")

## Espacios públicos

In [328]:
public_spaces = (
    gpd.read_file(data_path / "esp_pub/esp_pub.shp")
    .to_crs("EPSG:6372")
    .assign(
        radius=lambda df: np.sqrt(df.Sup_M2 / np.pi),
        geometry=lambda df: df.apply(lambda x: x.geometry.buffer(x.radius, resolution=64), axis=1)
    )
    .query("~TIPO.isin(['CAMELLONES', 'CUCHILLAS', 'GLORIETAS'])")
)

public_spaces_buffered = public_spaces.assign(
    geometry=lambda df: df["geometry"].buffer(1000, resolution=64)
)

rows["frac_ninos_mas_lejos_1km_parques"] = get_pop_not_intersects(joined, routes_buffered, pop_col="POB0_14")

# Escuelas

In [329]:
DENUE_TO_AMENITY_MAPPING = [
    # Salud
    {
        "name": "Consultorios médicos",
        "query": 'codigo_act.str.match("^621")',
    },
    {
        "name": "Hospital general",
        "query": 'codigo_act.str.match("^622")',
    },
    {
        "name": "Farmacia",
        "query": 'codigo_act.str.match("^46411")',
    },
    # Recreativo
    {
        "name": "Cine",
        "query": 'codigo_act.str.match("^51213")',
    },
    {
        "name": "Otros servicios recreativos",
        "query": 'codigo_act.str.match("^(71399|712|713)")',
    },
    {
        "name": "Clubs deportivos y de acondicionamiento físico",
        "query": 'codigo_act.str.match("^(71391|71394)")'
        
    },
    # Educación
    {
        "name": "Guarderia",
        "query": 'codigo_act.str.match("^6244")',
    },
    {
        "name": "Educación preescolar",
        "query": 'codigo_act.str.match("^61111")',
    },
    {
        "name": "Educación primaria",
        "query": 'codigo_act.str.match("^61112")'
    },
    {
        "name": "Educación secundaria",
        "query": 'codigo_act.str.match("^(61113|61114)")'
    },
    {
        "name": "Educación media superior",
        "query": 'codigo_act.str.match("^(61115|61116)")'
    },
    {
        "name": "Educación superior",
        "query": 'codigo_act.str.match("^(6112|6113)")'
    }
]


def process_occupancy(x: str) -> str:
    matches = re.match(r'([0-9]+) a ([0-9]+) personas', x.strip())
    return (int(matches.groups()[0]) + int(matches.groups()[1])) / 2 if matches else matches

In [330]:
denue_base = gpd.GeoDataFrame(
    (
        pd.read_csv(
            data_path / "denue.csv",
            encoding="latin1",
        )
        .query("municipio == 'Mexicali'")
        .assign(geometry=lambda x: gpd.points_from_xy(x.longitud, x.latitud))
        .drop(columns=["municipio"])
        .set_index("clee")
        .assign(
            per_ocu=lambda x: x.per_ocu.apply(process_occupancy),
            codigo_act=lambda x: x.codigo_act.astype(str)
        )
        .dropna(subset=["per_ocu"])
        .rename(columns={"per_ocu": "num_workers"})
        [["geometry", "codigo_act", "num_workers"]]
    ),
    crs="EPSG:4326",
    geometry="geometry"
).to_crs("EPSG:6372")

for elem in DENUE_TO_AMENITY_MAPPING:
    idx = denue_base.query(elem["query"]).index
    denue_base.loc[idx, "amenity"] = elem["name"]

denue_base = (
    denue_base
    .dropna(subset=["amenity"])
    .drop(columns=["codigo_act"])
)

In [331]:
schools = denue_base.query("amenity.isin(['Educación preescolar', 'Educación primaria', 'Educación secundaria', 'Educación media superior', 'Educación superior'])")
schools_buffered = schools.assign(
    geometry=lambda df: df["geometry"].buffer(1000, resolution=64)
)

rows["frac_personas_mas_lejos_1km_escuelas"] = get_pop_not_intersects(joined, schools_buffered, pop_col="POBTOT")

## Hospitales

In [332]:
hospitals = denue_base.query("amenity == 'Hospital general'")
hospitals_buffered = hospitals.assign(
    geometry=lambda df: df["geometry"].buffer(1000, resolution=64)
)
rows["frac_personas_mas_lejos_1km_hospitales"] = get_pop_not_intersects(joined, hospitals_buffered, pop_col="POBTOT")

## Hospitales y consultorios

In [333]:
hospitals_extra = denue_base.query("amenity.isin(['Hospital general', 'Consultorio médico', 'Farmacia'])")
hospitals_extra_buffered = hospitals_extra.assign(
    geometry=lambda df: df["geometry"].buffer(1000, resolution=64)
)
rows["frac_personas_mas_lejos_1km_hospitales_y_farmacias"] = get_pop_not_intersects(joined, hospitals_extra_buffered, pop_col="POBTOT")

In [334]:
out = pd.DataFrame.from_dict(rows, orient="index", columns=["value"]).reset_index(names="stat")
out.to_csv("./stats.csv", index=False)